<a href="https://colab.research.google.com/github/shivammehta007/NLPResearch/blob/master/Tutorials/Natural%20Language%20Processing/PyTorch%20Sentimental%20Analysis/First_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch 
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
SEED = 1234
torch.manual_seed(SEED)

In [0]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype=torch.float)

In [4]:
from torchtext import datasets
train_dataset , test_dataset = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 38.7MB/s]


In [5]:
len(train_dataset), len(test_dataset)

(25000, 25000)

In [0]:
import random

train_data, validation_data = train_dataset.split(random_state= random.seed(SEED))

In [7]:
len(train_data), len(validation_data)

(17500, 7500)

In [0]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(len(TEXT.vocab), len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(20))


25002 2
[('the', 202273), (',', 191724), ('.', 165509), ('and', 109826), ('a', 109039), ('of', 100566), ('to', 93435), ('is', 76237), ('in', 61513), ('I', 53946), ('it', 53635), ('that', 49166), ('"', 44415), ("'s", 42942), ('this', 42362), ('-', 37016), ('/><br', 35354), ('was', 35200), ('as', 30258), ('with', 29916)]


In [10]:
torch.cuda.is_available()

True

In [0]:
BATCH_SIZES= 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits( (train_data, validation_data, test_dataset ), batch_size=BATCH_SIZES, device=device)

In [0]:
class RNN(nn.Module):
    def __init__(self, input_dims, embedded_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dims, embedded_dim)
        self.rnn = nn.RNN(embedded_dim, hidden_dim)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
        hidden = self.dropout(hidden.squeeze(0))
        return self.fc(hidden)
        

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)


In [26]:
print('Number Of Trainable Parameters: {:,}'.format(sum(p.numel() for p in model.parameters())))

Number Of Trainable Parameters: 2,592,105


In [0]:
optimzer = optim.SGD(model.parameters(), lr=0.001)

In [0]:
loss = nn.BCEWithLogitsLoss()
model = model.to(device)
loss = loss.to(device)

In [0]:
def binary_accuracy(y_pred, y):
    rounded_pred = torch.round(torch.sigmoid(y_pred))
    correct = (rounded_pred == y).float()
    accuracy = correct.sum()/ len(y_pred)
    return accuracy
    

In [0]:
def train(model, iterator, optimizer, loss_function):
    epoch_accuracy = 0
    epoch_loss = 0
    
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        prediction = model(batch.text).squeeze(1)
        loss = loss_function(prediction, batch.label)
        accuracy = binary_accuracy(prediction, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_accuracy += accuracy.item()
        
    return epoch_loss / len(iterator) , epoch_accuracy / len(iterator)
        
        
        

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
def evaluate(model, iterator, loss_function):
    model.eval()
    
    epoch_loss = 0
    epoch_accuracy = 0
    
    with torch.no_grad():
        for batch in iterator:
            prediction = model(batch.text).squeeze(1)
            loss = loss_function(prediction, batch.label)
            acc = binary_accuracy(prediction, batch.label)
            
            epoch_loss += loss.item()
            epoch_accuracy += acc.item()
        
    return epoch_loss / len(iterator) , epoch_accuracy/ len(iterator)


In [33]:
EPOCHS = 10

best_valid_loss = float('inf')

best_state_dict = {}
for epoch in range(EPOCHS):
    start_time = time.time()
    
    train_loss, train_accuracy = train(model, train_iterator, optimzer, loss)

    print('Train Loss: {} , Train Accuracy: {}'.format(train_loss, train_accuracy))

    valid_loss, valid_accuracy = evaluate(model, valid_iterator, loss)
    
    print('Validation Loss: {}, Validation Accuracy: {}'.format(valid_loss, valid_accuracy))
    
    end_time = time.time()
    
    epoch_mins, epoch_sec = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_state_dict = model.state_dict()
        
    print('Epoch : {:2}, Epoch Time : {:2}m{:2}s'.format(epoch, epoch_mins, epoch_sec))


Train Loss: 0.6959577918487744 , Train Accuracy: 0.5013197342844775
Validation Loss: 0.6957829033924361, Validation Accuracy: 0.4880384888689397
Epoch :  0, Epoch Time :  0m24s
Train Loss: 0.6963565549711241 , Train Accuracy: 0.4898657457236826
Validation Loss: 0.6949578382201114, Validation Accuracy: 0.490024717682499
Epoch :  1, Epoch Time :  0m24s
Train Loss: 0.6945790039361829 , Train Accuracy: 0.5047005345351505
Validation Loss: 0.695247793096607, Validation Accuracy: 0.4887005651401261
Epoch :  2, Epoch Time :  0m24s
Train Loss: 0.6953943062872783 , Train Accuracy: 0.4976456596033417
Validation Loss: 0.6954819317591392, Validation Accuracy: 0.4873764125977532
Epoch :  3, Epoch Time :  0m24s
Train Loss: 0.6952920377689557 , Train Accuracy: 0.4947292102079322
Validation Loss: 0.6950839318461337, Validation Accuracy: 0.4893626414113126
Epoch :  4, Epoch Time :  0m24s
Train Loss: 0.6954676933097144 , Train Accuracy: 0.4964318301555884
Validation Loss: 0.6949649301625914, Validation A

In [0]:
# Check on Testing Set 
model.load_state_dict(best_state_dict)
testing_loss, testing_accuracy = evaluate(model, test_iterator, loss)

In [39]:
print('Test Loss : {}, Test Accuracy : {:.2f}%'.format(testing_loss, testing_accuracy*100))

Test Loss : 0.6854497983937373, Test Accuracy : 59.84%
